In [1]:
from compound_db_utils import settings
from compound_db_utils.data_loaders import fetch_learning_data

data = fetch_learning_data(
        ['MI-T-722a16f1']
        , bioacitivities_cols=('activity_comment', 'value')
        , col_names_map={
            settings.COMPOUNDS_TABLE + '_smiles' : 'smiles'
            , settings.BIOACTIVITIES_TABLE + '_value' : 'ic50_experiment'
            , settings.BIOACTIVITIES_TABLE + '_activity_comment' : 'comment'
        }
        , create_rdkit_mols=False
        , compute_descriptors=False
    )

/home/sichom/Software/virtualenvs/miniconda3/envs/MI_ADM/lib/python3.5/site-packages/rdkit/Chem/PeriodicTable.py:131: FutureWarning: split() requires a non-empty pattern match.
  splitLine = blankExpr.split(line)


In [2]:
from IPython.display import HTML
HTML(data[:5].to_html())

,comment,ic50_experiment,smiles
0,Unspecified,4.7,CCc1c(O)ccc2c1OC(c1ccc(OCCN3CCCCC3)cc1)C(c1ccc(O)cc1)S2
1,Unspecified,24.8,Cc1ccc(S(=O)(=O)N2CCc3cc(O)ccc3C2c2ccc(OCCN3CCCC3)cc2)cc1
2,Unspecified,6.0,CC1=C(c2ccc(O)cc2)C(=O)c2ccc(O)cc21
3,Unspecified,1.3,CC(COc1ccc(C2Oc3ccc(O)cc3SC2c2ccc(O)cc2)cc1)N1CCCC1
4,Unspecified,1.3,CC(COc1ccc(C2Oc3ccc(O)cc3SC2c2ccc(O)cc2)cc1)N1CCCC1
